# ADMIN DASHBOARD

In [4]:
import psycopg2
from datetime import datetime
import re
import datetime



# Admin credentials
ADMIN_USERNAME = "surya"
ADMIN_PASSWORD = "22069"  

# Establish connection to PostgreSQL
def connect_db():
    try:
        conn = psycopg2.connect(
            host="localhost",
            port="5432",  
            database="hospital_management",  
            user="postgres",  
            password="goli00"  
        )
        conn.autocommit = True  
        return conn
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None

# Admin authentication
def admin_login():
    print("Admin Portal Login")
    username = input("Enter username: ")
    password = input("Enter password: ")

    if username == ADMIN_USERNAME and password == ADMIN_PASSWORD:
        print("Login successful!")
        return True
    else:
        print("Invalid username or password.")
        return False

def delete_patient(cursor):
    try:
        # Check if there are any patients in the database
        cursor.execute("SELECT * FROM patients")
        patients = cursor.fetchall()

        if not patients:
            print("Can't delete.")
            return

        # Proceed with deletion if patients exist
        patient_id = int(input("Enter patient ID to delete: "))
        
        # Check if the patient_id exists
        cursor.execute("SELECT * FROM patients WHERE patient_id = %s", (patient_id,))
        patient = cursor.fetchone()

        if patient:
            # If the patient exists, proceed to delete
            query = "DELETE FROM patients WHERE patient_id = %s"
            cursor.execute(query, (patient_id,))
            print("Patient deleted successfully!")
        else:
            print(f"Patient with ID {patient_id} does not exist.")

    except Exception as e:
        print(f"Error deleting patient: {e}")

def view_patients(cursor):
    try:
        query = "SELECT * FROM patients"
        cursor.execute(query)
        patients = cursor.fetchall()

        if not patients:
            print("No patients found.")
            return

        print("\nPatients:")
        print(f"{'Patient ID':<12}{'Name':<20}{'Age':<5}{'Gender':<10}{'Contact Number':<15}{'Address':<25}{'Admission Date':<15}")
        print("-" * 100)

        for patient in patients:
            patient_id, name, age, gender, contact_number, address, admission_date = patient
            admission_date_str = admission_date.strftime('%Y-%m-%d') if admission_date else "N/A"
            print(f"{str(patient_id):<12}{name:<20}{str(age):<5}{gender:<10}{contact_number:<15}{address:<25}{admission_date_str:<15}")
    except Exception as e:
        print(f"Error viewing patients: {e}")

        
# Billing management functions
def add_bill(cursor):
    try:
        # Fetch patients with completed appointments
        query = """
            SELECT p.patient_id, p.name FROM patients p
            JOIN appointments a ON p.patient_id = a.patient_id
            WHERE a.status = 'completed'
        """
        cursor.execute(query)
        completed_patients = cursor.fetchall()

        if not completed_patients:
            print("No patients with completed appointments found.")
            return

        print("\nPatients with completed appointments:")
        print(f"{'Patient ID':<12}{'Name':<20}")
        print("-" * 40)

        for patient in completed_patients:
            patient_id, name = patient
            print(f"{str(patient_id):<12}{name:<20}")

        patient_id = int(input("Enter the patient ID to add a bill: "))
        patient_name = input("Enter the patient's name: ")
        amount = float(input("Enter bill amount: "))
        
        # Get the current date and time
        bill_date = datetime.now()

        query = """
            INSERT INTO bills (patient_id, patient_name, amount, billing_date)
            VALUES (%s, %s, %s, %s)
        """
        cursor.execute(query, (patient_id, patient_name, amount, bill_date))
        print("Bill added successfully!")
    except Exception as e:
        print(f"Error adding bill: {e}")
        


def view_bills(cursor):
    try:
        query = "SELECT * FROM bills"
        cursor.execute(query)
        bills = cursor.fetchall()

        if not bills:
            print("No bills found for any patients.")
            return

        print("\nBills:")
        print(f"{'Bill ID':<10}{'Patient ID':<12}{'Patient Name':<20}{'Amount':<10}{'Billing Date':<15}{'Status':<10}")
        print("-" * 80)

        for bill in bills:
            # Unpack the 6 columns from the bill record
            bill_id, patient_id, patient_name, amount, billing_date, status = bill
            
            # Check the type of billing_date and format accordingly
            if isinstance(billing_date, str):
                billing_date_str = billing_date  # Already a string
            else:
                billing_date_str = billing_date.strftime('%Y-%m-%d') if billing_date else "N/A"  # Convert to string if it's a datetime object

            print(f"{str(bill_id):<10}{str(patient_id):<12}{patient_name:<20}{str(amount):<10}{billing_date_str:<15}{status:<10}")
    except Exception as e:
        print(f"Error viewing bills: {e}")
        

def complete_bill(cursor):
    try:
        # Fetch bills that are not marked as "payment done"
        query = "SELECT * FROM bills WHERE status != 'payment done'"
        cursor.execute(query)
        bills = cursor.fetchall()

        if not bills:
            print("No pending bills to complete.")
            return

        print("\nPending Bills:")
        print(f"{'Bill ID':<10}{'Patient ID':<12}{'Patient Name':<20}{'Amount':<10}{'Billing Date':<15}{'Status':<10}")
        print("-" * 80)

        for bill in bills:
            bill_id, patient_id, patient_name, amount, billing_date, status = bill
            
            if isinstance(billing_date, str):
                billing_date_str = billing_date  # Already a string
            else:
                billing_date_str = billing_date.strftime('%Y-%m-%d') if billing_date else "N/A"  # Convert to string if it's a datetime object

            print(f"{str(bill_id):<10}{str(patient_id):<12}{patient_name:<20}{str(amount):<10}{billing_date_str:<15}{status:<10}")

            bill_id_to_complete = input("\nEnter the Bill ID to mark as 'payment done' (or press Enter to cancel): ")

        if not bill_id_to_complete:
            print("No bill completed.")
            return

        # Update the selected bill's status to 'payment done'
        update_query = "UPDATE bills SET status = 'payment done' WHERE bill_id = %s"
        cursor.execute(update_query, (bill_id_to_complete,))
        cursor.connection.commit()  # Use the cursor's connection to commit

        print(f"Bill ID {bill_id_to_complete} has been marked as 'payment done'.")
    except Exception as e:
        print(f"Error completing bill: {e}")
        

def manage_doctors(cursor):
    while True:
        print("\nDoctor Management")
        print("1. Add Doctor")
        print("2. Update Doctor")
        print("3. Delete Doctor")
        print("4. View Doctors")
        print("5. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            name = input("Enter doctor's name: ")
            specialty = input("Enter doctor's specialty: ")
            contact_number = input("Enter contact number: ")

            query = """
                INSERT INTO doctors (name, specialty, contact_number)
                VALUES (%s, %s, %s)
            """
            cursor.execute(query, (name, specialty, contact_number))
            print("Doctor added successfully!")

        elif choice == '2':
            cursor.execute("SELECT * FROM doctors")
            rows = cursor.fetchall()

            if rows:
                print("\nDoctor List:")
                print(f"{'ID':<5} {'Name':<20} {'Specialty':<20} {'Contact Number':<15}")
                print("-" * 60)
                for row in rows:
                    print(f"{row[0]:<5} {row[1]:<20} {row[2]:<20} {row[3]:<15}")
            else:
                print("No doctors found.")

            doctor_id = int(input("Enter doctor ID to update: "))
            cursor.execute("SELECT * FROM doctors WHERE doctor_id = %s", (doctor_id,))
            doctor = cursor.fetchone()
            
            if doctor:
                name = input("Enter new name: ")
                specialty = input("Enter new specialty: ")
                contact_number = input("Enter new contact number: ")
                query = """UPDATE doctors SET name = %s, specialty = %s, contact_number = %s WHERE doctor_id = %s"""
                cursor.execute(query, (name, specialty, contact_number, doctor_id))
                print("Doctor updated successfully!")
                cursor.execute("SELECT * FROM doctors")
                rows = cursor.fetchall()

                if rows:
                    print("\nDoctor List:")
                    print(f"{'ID':<5} {'Name':<20} {'Specialty':<20} {'Contact Number':<15}")
                    print("-" * 60)
                    for row in rows:
                        print(f"{row[0]:<5} {row[1]:<20} {row[2]:<20} {row[3]:<15}")
                else:
                    print("No doctors found.")
                    
            else:
                    print(f"Doctor with ID {doctor_id} does not exist.")


        elif choice == '3':
            try:
                cursor.execute("SELECT * FROM doctors")
                rows = cursor.fetchall()

                if rows:
                    print("\nDoctor List:")
                    print(f"{'ID':<5} {'Name':<20} {'Specialty':<20} {'Contact Number':<15}")
                    print("-" * 60)
                    for row in rows:
                        print(f"{row[0]:<5} {row[1]:<20} {row[2]:<20} {row[3]:<15}")
                else:
                    print("No doctors found.")

                doctor_id = int(input("Enter doctor ID to delete: "))
                cursor.execute("SELECT * FROM doctors WHERE doctor_id = %s", (doctor_id,))
                doctor = cursor.fetchone()
                   
                if doctor:
                    query = "DELETE FROM doctors WHERE doctor_id = %s"
                    cursor.execute(query, (doctor_id,))
                    print("Doctor deleted successfully!")
                    cursor.execute("SELECT * FROM doctors")
                    rows = cursor.fetchall()

                    if rows:
                        print("\nDoctor List:")
                        print(f"{'ID':<5} {'Name':<20} {'Specialty':<20} {'Contact Number':<15}")
                        print("-" * 60)
                        for row in rows:
                            print(f"{row[0]:<5} {row[1]:<20} {row[2]:<20} {row[3]:<15}")
                    else:
                        print("No doctors found.")
                    
                else:
                    print(f"Doctor with ID {doctor_id} does not exist.")

            except Exception as e:
                print(f"Error deleting doctor: {e}")

        elif choice == '4':
            cursor.execute("SELECT * FROM doctors")
            rows = cursor.fetchall()

            if rows:
                print("\nDoctor List:")
                print(f"{'ID':<5} {'Name':<20} {'Specialty':<20} {'Contact Number':<15}")
                print("-" * 60)
                for row in rows:
                    print(f"{row[0]:<5} {row[1]:<20} {row[2]:<20} {row[3]:<15}")
            else:
                print("No doctors found.")

        elif choice == '5':
            break
        else:
            print("Invalid choice. Please enter a valid option.")

            
# Staff management functions
def register_staff(cursor):
    try:
        while True:
            name = input("Enter staff name: ")
            if re.match(r"^[a-zA-Z\s]+$", name):
                break
            print("Invalid name. Please enter alphabetic characters only.")

        role = input("Enter staff role: ")

        while True:
            contact_number = input("Enter contact number: ")
            if re.match(r"^\d{10}$", contact_number):
                break
            print("Invalid contact number. Please enter a 10-digit number.")

        department = input("Enter department: ")

        query = """
            INSERT INTO staff (name, role, contact_number, department)
            VALUES (%s, %s, %s, %s)
        """
        cursor.execute(query, (name, role, contact_number, department))
        print("Staff registration successful!")
    except Exception as e:
        print(f"Error registering staff: {e}")

def update_staff(cursor):
    try:
        # Check if there are any staff members
        cursor.execute("SELECT * FROM staff")
        staff_list = cursor.fetchall()

        if not staff_list:
            print("No staff registered.")
            return

        # Display all registered staff
        print("\nRegistered Staff:")
        print(f"{'ID':<5} {'Name':<20} {'Role':<15} {'Contact':<15} {'Department':<15}")
        print("-" * 70)
        for staff in staff_list:
            print(f"{staff[0]:<5} {staff[1]:<20} {staff[2]:<15} {staff[3]:<15} {staff[4]:<15}")
        
        # Ask for the staff ID to update
        staff_id = int(input("\nEnter staff ID to update: "))

        # Check if the staff_id exists
        cursor.execute("SELECT * FROM staff WHERE staff_id = %s", (staff_id,))
        staff = cursor.fetchone()

        if not staff:
            print(f"Staff with ID {staff_id} does not exist.")
            return

        # Proceed with updating staff details
        name = input("Enter new staff name: ")
        role = input("Enter new role: ")
        contact_number = input("Enter new contact number: ")
        department = input("Enter new department: ")

        query = """
            UPDATE staff
            SET name = %s, role = %s, contact_number = %s, department = %s
            WHERE staff_id = %s
        """
        cursor.execute(query, (name, role, contact_number, department, staff_id))
        print("Staff details updated successfully!")

    except Exception as e:
        print(f"Error updating staff: {e}")

        
def delete_staff(cursor):
    try:
        # Check if any staff members are registered
        cursor.execute("SELECT * FROM staff")
        staff_list = cursor.fetchall()

        if not staff_list:
            print("No registered staff to delete.")
            return

        # Display all registered staff
        print("\nRegistered Staff:")
        print(f"{'ID':<5} {'Name':<20} {'Role':<15} {'Contact':<15} {'Department':<15}")
        print("-" * 70)
        for staff in staff_list:
            print(f"{staff[0]:<5} {staff[1]:<20} {staff[2]:<15} {staff[3]:<15} {staff[4]:<15}")
        
        # Proceed to ask for staff ID to delete
        staff_id = int(input("\nEnter staff ID to delete: "))

        # Check if the staff_id exists
        cursor.execute("SELECT * FROM staff WHERE staff_id = %s", (staff_id,))
        staff = cursor.fetchone()

        if staff:
            # If the staff member exists, proceed to delete
            query = "DELETE FROM staff WHERE staff_id = %s"
            cursor.execute(query, (staff_id,))
            print("Staff deleted successfully!")
        else:
            print(f"Staff with ID {staff_id} does not exist.")

    except Exception as e:
        print(f"Error deleting staff: {e}")



def view_staff(cursor):
    try:
        query = "SELECT * FROM staff"
        cursor.execute(query)
        staff = cursor.fetchall()

        if not staff:
            print("No staff found.")
            return

        print("\nStaff:")
        print(f"{'Staff ID':<12}{'Name':<20}{'Role':<15}{'Contact':<15}{'Department':<20}")
        print("-" * 80)

        for member in staff:
            staff_id, name, role, contact_number, department = member
            print(f"{str(staff_id):<12}{name:<20}{role:<15}{contact_number:<15}{department:<20}")
    except Exception as e:
        print(f"Error viewing staff: {e}")

def view_pending_leaves(cursor):
    try:
        query = "SELECT * FROM leaves WHERE status='pending'"
        cursor.execute(query)
        leaves = cursor.fetchall()

        if not leaves:
            print("No leave applications found.")
            return

        # Print headers with appropriate spacing
        print("\nLeave Applications:")
        print(f"{'Application ID':<15}{'Doctor ID':<12}{'Leave Start':<15}{'Leave End':<15}{'Status':<10}")
        print("-" * 70)

        # Loop through each leave record and print with proper alignment
        for leave in leaves:
            application_id, doctor_id, leave_start, leave_end, status = leave
            # Ensure the dates are in string format and correctly aligned
            leave_start_str = leave_start.strftime('%Y-%m-%d') if isinstance(leave_start, (datetime.date, datetime.datetime)) else str(leave_start)
            leave_end_str = leave_end.strftime('%Y-%m-%d') if isinstance(leave_end, (datetime.date, datetime.datetime)) else str(leave_end)
            print(f"{str(application_id):<15}{str(doctor_id):<12}{leave_start_str:<15}{leave_end_str:<15}{status:<10}")
            process_leave_application(cursor)
    except Exception as e:
        print(f"Error viewing leave applications: {e}")
        
def process_leave_application(cursor):
    try:
        application_id = input("Enter the application ID to process (or press Enter to skip): ")

        if not application_id:
            print("No leave processed.")
            return

        action = input("Enter 'accept' to approve the leave, 'reject' to reject it, or press Enter to skip: ").lower()

        if action == 'accept':
            update_query = "UPDATE leaves SET status = 'accepted' WHERE leave_id = %s"
            cursor.execute(update_query, (application_id,))
            print("Leave accepted.")
        elif action == 'reject':
            update_query = "UPDATE leaves SET status = 'rejected' WHERE leave_id = %s"
            cursor.execute(update_query, (application_id,))
            print("Leave rejected.")
        else:
            print("No changes made.")
    except Exception as e:
        print(f"Error processing leave application: {e}")

def view_accepted_rejected_leaves(cursor):
    try:
        query = "SELECT * FROM leaves WHERE status IN ('accepted', 'rejected')"
        cursor.execute(query)
        processed_leaves = cursor.fetchall()

        if not processed_leaves:
            print("No accepted or rejected leave applications.")
            return

        print("\nProcessed Leave Applications:")
        print(f"{'Application ID':<15}{'Doctor ID':<12}{'Leave Start':<15}{'Leave End':<15}{'Status':<10}")
        print("-" * 70)

        for leave in processed_leaves:
            application_id, doctor_id, leave_start, leave_end, status = leave
            # Ensure the dates are in string format and correctly aligned
            leave_start_str = leave_start.strftime('%Y-%m-%d') if isinstance(leave_start, (datetime.date, datetime.datetime)) else str(leave_start)
            leave_end_str = leave_end.strftime('%Y-%m-%d') if isinstance(leave_end, (datetime.date, datetime.datetime)) else str(leave_end)
            print(f"{str(application_id):<15}{str(doctor_id):<12}{leave_start_str:<15}{leave_end_str:<15}{status:<10}")
    except Exception as e:
        print(f"Error viewing accepted and rejected leave applications: {e}")

# Manage leaves
def manage_leaves(cursor):
    while True:
        print("\nLeave Management")
        print("1. View and Process Pending Leaves")
        print("2. View Processed (Accepted/Rejected) Leaves")
        print("3. Go Back")

        choice = input("Enter your choice: ")

        if choice == '1':
            view_pending_leaves(cursor)
        elif choice == '2':
            view_accepted_rejected_leaves(cursor)
        elif choice == '3':
            break
        else:
            print("Invalid choice. Please enter a valid option.")


# Main menu function
def main_menu(cursor):
    while True:
        print("\nHospital Management System")
        print("1. Manage Doctors")
        print("2. Manage Patients")
        print("3. Manage Staff")
        print("4. Manage Billing")
        print("5. Manage Leaves")
        print("6. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            manage_doctors(cursor)
        elif choice == '2':
            view_patients(cursor)
            while True:
                print("\nPatient Management")
                print("1. Delete Patient")
                print("2. Go Back")
                
                patient_choice = input("Enter your choice: ")
                
                if patient_choice == '1':
                    view_patients(cursor)
                    delete_patient(cursor)
                elif patient_choice == '2':
                    break
                else:
                    print("Invalid choice. Please enter a valid option.")
        elif choice == '3':
            while True:
                print("\nStaff Management")
                print("1. Register Staff")
                print("2. Update Staff")
                print("3. Delete Staff")
                print("4. View Staff")
                print("5. Go Back")

                staff_choice = input("Enter your choice: ")

                if staff_choice == '1':
                    register_staff(cursor)
                elif staff_choice == '2':
                    update_staff(cursor)
                elif staff_choice == '3':
                    delete_staff(cursor)
                elif staff_choice == '4':
                    view_staff(cursor)
                elif staff_choice == '5':
                    break
                else:
                    print("Invalid choice. Please enter a valid option.")
        elif choice == '4':
            while True:
                print("\nBilling Management")
                print("1. Add Bill")
                print("2. View Bills")
                print("3. Complete Bills")
                print("4. Go Back")

                bill_choice = input("Enter your choice: ")

                if bill_choice == '1':
                    add_bill(cursor)
                elif bill_choice == '2':
                    view_bills(cursor)
                elif bill_choice == '3':
                    complete_bill(cursor)
                elif bill_choice == '4':
                    break
                else:
                    print("Invalid choice. Please enter a valid option.")
        elif choice == '5':
            manage_leaves(cursor)
        elif choice == '6':
            break
        else:
            print("Invalid choice. Please enter a valid option.")

# Main execution
def main():
    conn = connect_db()
    if conn is None:
        return

    cursor = conn.cursor()
    if admin_login():
        main_menu(cursor)

    cursor.close()
    conn.close()

if __name__ == "__main__":
    main()


Admin Portal Login
Enter username: surya
Enter password: 22069
Login successful!

Hospital Management System
1. Manage Doctors
2. Manage Patients
3. Manage Staff
4. Manage Billing
5. Manage Leaves
6. Exit
Enter your choice: 5

Leave Management
1. View and Process Pending Leaves
2. View Processed (Accepted/Rejected) Leaves
3. Go Back
Enter your choice: 3

Hospital Management System
1. Manage Doctors
2. Manage Patients
3. Manage Staff
4. Manage Billing
5. Manage Leaves
6. Exit
Enter your choice: 6


# DOCTOR DASHBOARD

In [5]:
import psycopg2
from datetime import datetime, timedelta

# Establish connection to PostgreSQL
def connect_db():
    try:
        conn = psycopg2.connect(
            host="localhost",
            port="5432",  # Update this to your port
            database="hospital_management",  # Ensure this is your database name
            user="postgres",  # Your PostgreSQL username
            password="goli00"  # Your PostgreSQL password
        )
        conn.autocommit = True  # Ensure changes are committed automatically
        return conn
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None

# Function to verify doctor credentials
def verify_doctor(cur, doctor_id, doctor_name):
    query = "SELECT * FROM doctors WHERE doctor_id = %s AND name = %s"
    cur.execute(query, (doctor_id, doctor_name))
    return cur.fetchone() is not None

# Function to apply for leave
def apply_leave(cur, doctor_id):
    # Check for existing pending leave
    query = "SELECT status FROM leaves WHERE doctor_id = %s AND status = 'pending'"
    cur.execute(query, (doctor_id,))
    pending_leave = cur.fetchone()

    if pending_leave:
        print("You have a pending leave application. You cannot apply for a new leave until it is processed.")
        return

    leave_start_date = input("Enter leave start date (YYYY-MM-DD): ")
    leave_end_date = input("Enter leave end date (YYYY-MM-DD): ")

    # Insert leave application into the database
    query = """
        INSERT INTO leaves (doctor_id, leave_start_date, leave_end_date, status)
        VALUES (%s, %s, %s, 'pending')
    """
    cur.execute(query, (doctor_id, leave_start_date, leave_end_date))
    print("Leave application submitted successfully!")

# Function to withdraw leave
def withdraw_leave(cur, doctor_id):
    query = "SELECT leave_id FROM leaves WHERE doctor_id = %s AND status = 'pending'"
    cur.execute(query, (doctor_id,))
    pending_leaves = cur.fetchall()

    if not pending_leaves:
        print("No pending leave applications to withdraw.")
        return

    print("Pending Leave Applications:")
    for leave in pending_leaves:
        print(f"Leave ID: {leave[0]}")

    leave_id = int(input("Enter Leave ID to withdraw (0 to cancel): "))
    if leave_id == 0:
        return

    query = "DELETE FROM leaves WHERE leave_id = %s AND doctor_id = %s AND status = 'pending'"
    cur.execute(query, (leave_id, doctor_id))
    print(f"Leave ID {leave_id} withdrawn successfully!")

# Function to view leave status
def view_leave_status(cur, doctor_id):
    query = "SELECT leave_start_date, leave_end_date, status FROM leaves WHERE doctor_id = %s"
    cur.execute(query, (doctor_id,))
    leaves = cur.fetchall()

    if not leaves:
        print("No leave applications found.")
        return

    print(f"{'Start Date':<15}{'End Date':<15}{'Status':<15}")
    print("-" * 45)
    for leave in leaves:
        start_date = leave[0].strftime("%Y-%m-%d") if leave[0] else 'N/A'  # Format date
        end_date = leave[1].strftime("%Y-%m-%d") if leave[1] else 'N/A'  # Format date
        status = leave[2] if leave[2] else 'N/A'  # Handle None
        print(f"{start_date:<15}{end_date:<15}{status:<15}")

# Function to view and manage appointments
def manage_appointments(cur, doctor_id):
    while True:
        print("\nAppointment Management")
        print("Fetching appointments...")

        # Fetch appointments for the specific doctor
        query = """
            SELECT a.appointment_id, p.name, a.appointment_date, a.appointment_time, a.status
            FROM appointments a
            JOIN patients p ON a.patient_id = p.patient_id
            WHERE a.doctor_id = %s
        """
        cur.execute(query, (doctor_id,))
        appointments = cur.fetchall()

        if not appointments:
            print("No appointments found.")
            return

        # Display appointments
        print(f"{'Appointment ID':<15}{'Patient Name':<20}{'Date':<15}{'Time':<10}{'Status':<15}")
        print("-" * 70)
        for appointment in appointments:
            appointment_id, patient_name, appointment_date, appointment_time, status = appointment
            # Format date and time for better readability
            formatted_date = appointment_date.strftime("%Y-%m-%d")  # Assuming appointment_date is a date object
            formatted_time = appointment_time.strftime("%H:%M")  # Assuming appointment_time is a time object
            print(f"{appointment_id:<15}{patient_name:<20}{formatted_date:<15}{formatted_time:<10}{status:<15}")

        appointment_id = int(input("Enter the Appointment ID to manage (0 to exit): "))
        if appointment_id == 0:
            break

        # Check the status of the selected appointment
        query = "SELECT status FROM appointments WHERE appointment_id = %s"
        cur.execute(query, (appointment_id,))
        appointment_status = cur.fetchone()

        if appointment_status is None:
            print("Appointment not found.")
            continue
        
        appointment_status = appointment_status[0]

        if appointment_status == 'completed':
            print("This appointment has already been completed and cannot be modified.")
            continue
        
        action = input("Do you want to (1) Complete, (2) Accept, or (3) Postpone the appointment? ")

        if action == '1':
            # Update the appointment status to "completed"
            query = "UPDATE appointments SET status = 'completed' WHERE appointment_id = %s"
            cur.execute(query, (appointment_id,))
            print("Appointment status updated to completed.")
        
        elif action == '2':
            # Update the appointment status to "accepted"
            query = "UPDATE appointments SET status = 'accepted' WHERE appointment_id = %s"
            cur.execute(query, (appointment_id,))
            print("Appointment status updated to accepted.")

        elif action == '3':
            days = int(input("Enter number of days to postpone: "))
            new_date = datetime.now().date() + timedelta(days=days)  # Use timedelta here
            # Update the appointment status and date
            query = """
                UPDATE appointments 
                SET status = %s, appointment_date = %s 
                WHERE appointment_id = %s
            """
            new_status = f'postponed for {days} days'
            cur.execute(query, (new_status, new_date, appointment_id))
            print(f"Appointment postponed to {new_date} with status '{new_status}'.")
        else:
            print("Invalid action.")

# Function to create treatment plans for patients
def manage_treatment_plans(cur, doctor_id):
    # Fetch confirmed appointments for the specific doctor
    query = """
        SELECT a.patient_id, p.name
        FROM appointments a
        JOIN patients p ON a.patient_id = p.patient_id
        WHERE a.doctor_id = %s AND a.status = 'completed'
    """
    cur.execute(query, (doctor_id,))
    patients = cur.fetchall()

    if not patients:
        print("No confirmed appointments found for treatment plans.")
        return

    print("Patients with confirmed appointments:")
    print(f"{'Patient ID':<15}{'Patient Name':<20}")
    print("-" * 35)
    for patient in patients:
        print(f"{patient[0]:<15}{patient[1]:<20}")

    while True:
        patient_id = int(input("Enter Patient ID to create a treatment plan (0 to exit): "))
        if patient_id == 0:
            break

        # Check if the patient has an appointment with the doctor
        query = """
            SELECT a.appointment_id 
            FROM appointments a 
            WHERE a.patient_id = %s AND a.doctor_id = %s AND a.status = 'completed'
        """
        cur.execute(query, (patient_id, doctor_id))
        appointment = cur.fetchone()

        if not appointment:
            print("Patient does not have a completed appointment with you. Please select a patient with an existing appointment.")
            continue

        # Fetch patient details
        query = "SELECT name FROM patients WHERE patient_id = %s"
        cur.execute(query, (patient_id,))
        patient = cur.fetchone()

        if patient:
            patient_name = patient[0]
            print(f"Creating treatment plan for {patient_name} (ID: {patient_id})")

            diagnosis = input("Enter diagnosis: ")
            medications = input("Enter medications: ")
            procedures = input("Enter procedures: ")
            treatment_date = input("Enter treatment date (YYYY-MM-DD): ")

            # Insert treatment plan into the database
            query = """
                INSERT INTO treatment_plans (patient_id, doctor_id, diagnosis, medications, procedures, treatment_date)
                VALUES (%s, %s, %s, %s, %s, %s)
            """
            cur.execute(query, (patient_id, doctor_id, diagnosis, medications, procedures, treatment_date))
            print("Treatment plan added successfully!")
        else:
            print("Patient not found.")

# Main function for doctor dashboard
def doctor_dashboard():
    conn = connect_db()
    if not conn:
        print("Unable to establish connection. Exiting...")
        return

    cur = conn.cursor()
    
    print("Welcome to Doctor Dashboard")
    doctor_id = input("Enter Doctor ID: ")
    doctor_name = input("Enter Doctor Name: ")

    if not verify_doctor(cur, doctor_id, doctor_name):
        print("Invalid Doctor ID or Name.")
        return

    while True:
        print("\n1. Manage Appointments")
        print("2. Create Treatment Plans")
        print("3. Apply for Leave")
        print("4. Withdraw Leave")
        print("5. View Leave Status")
        print("6. Exit")
        
        choice = input("Enter your choice: ")
        if choice == '1':
            manage_appointments(cur, doctor_id)
        elif choice == '2':
            manage_treatment_plans(cur, doctor_id)
        elif choice == '3':
            apply_leave(cur, doctor_id)
        elif choice == '4':
            withdraw_leave(cur, doctor_id)
        elif choice == '5':
            view_leave_status(cur, doctor_id)
        elif choice == '6':
            break
        else:
            print("Invalid choice. Please try again.")

    cur.close()
    conn.close()

# Run the doctor dashboard
if __name__ == "__main__":
    doctor_dashboard()


Welcome to Doctor Dashboard
Enter Doctor ID: 4
Enter Doctor Name: Dr. Surya

1. Manage Appointments
2. Create Treatment Plans
3. Apply for Leave
4. Withdraw Leave
5. View Leave Status
6. Exit
Enter your choice: 6


# PATIENT DASHBOARD

In [6]:
import psycopg2
import datetime
import re
from datetime import datetime
from datetime import datetime, date



# Establish connection to PostgreSQL
def connect_db():
    try:
        conn = psycopg2.connect(
            host="localhost",
            port="5432",  # Update this to your port
            database="hospital_management",  # Ensure this is your database name
            user="postgres",  # Your PostgreSQL username
            password="goli00"  # Your PostgreSQL password
        )
        conn.autocommit = True  # Ensure changes are committed automatically
        return conn
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None

# Register a new patient
def register_patient(cursor):
    try:
        while True:
            name = input("Enter your name: ")
            if re.match(r"^[a-zA-Z\s]+$", name):
                break
            print("Invalid name. Please enter alphabetic characters only.")

        while True:
            age = input("Enter your age: ")
            if age.isdigit() and 0 < int(age) < 120:
                age = int(age)
                break
            print("Invalid age. Please enter a valid number between 1 and 120.")

        gender = input("Enter your gender: ")

        while True:
            phone = input("Enter your contact number: ")
            if re.match(r"^\d{10}$", phone):
                break
            print("Invalid contact number. Please enter a 10-digit number.")

        address = input("Enter your address: ")

        while True:
            admission_date = input("Enter your admission date (YYYY-MM-DD): ")
            try:
                datetime.strptime(admission_date, '%Y-%m-%d')
                break
            except ValueError:
                print("Invalid date format. Please enter in YYYY-MM-DD format.")

        query = """
            INSERT INTO patients (name, age, gender, contact_number, address, admission_date)
            VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(query, (name, age, gender, phone, address, admission_date))
        print("Registration successful!")
        cursor.execute("SELECT patient_id FROM patients WHERE contact_number = %s", (phone,))
        patient_id = cursor.fetchone()

        if patient_id:
            print(f"Patient ID: {patient_id[0]}, Name: {name}")
        else:
            print("Registration successful, but patient ID could not be retrieved.")
    except Exception as e:
        print(f"Error registering patient: {e}")

# Patient login: Validate patient ID and name
def login_patient(cursor):
    try:
        patient_id = input("Enter your patient ID: ")
        name = input("Enter your name: ")

        query = "SELECT * FROM patients WHERE patient_id = %s AND name = %s"
        cursor.execute(query, (patient_id, name))
        patient = cursor.fetchone()

        if patient:
            print(f"Welcome back, {name}!")
            return patient
        else:
            print("Invalid patient ID or name.")
            return None
    except Exception as e:
        print(f"Error logging in patient: {e}")
        return None

# Update patient details
def update_patient(cursor, patient_id):
    try:
        while True:
            name = input("Enter new name: ")
            if re.match(r"^[a-zA-Z\s]+$", name):
                break
            print("Invalid name. Please enter alphabetic characters only.")

        while True:
            age = input("Enter new age: ")
            if age.isdigit() and 0 < int(age) < 120:
                age = int(age)
                break
            print("Invalid age. Please enter a valid number between 1 and 120.")

        gender = input("Enter new gender: ")

        while True:
            phone = input("Enter new contact number: ")
            if re.match(r"^\d{10}$", phone):
                break
            print("Invalid contact number. Please enter a 10-digit number.")

        address = input("Enter new address: ")

        while True:
            admission_date = input("Enter new admission date (YYYY-MM-DD): ")
            try:
                datetime.strptime(admission_date, '%Y-%m-%d')
                break
            except ValueError:
                print("Invalid date format. Please enter in YYYY-MM-DD format.")

        query = """
            UPDATE patients
            SET name = %s, age = %s, gender = %s, contact_number = %s, address = %s, admission_date = %s
            WHERE patient_id = %s
        """
        cursor.execute(query, (name, age, gender, phone, address, admission_date, patient_id))
        print("Patient details updated successfully!")
    except Exception as e:
        print(f"Error updating patient: {e}")

# Display all doctors and book an appointment
def book_appointment(cursor, patient_id):
    try:
        query = "SELECT doctor_id, name, specialty FROM doctors"
        cursor.execute(query)
        doctors = cursor.fetchall()

        if not doctors:
            print("No doctors available.")
            return

        print("\nAvailable Doctors:")
        print(f"{'Doctor ID':<12}{'Name':<20}{'Specialty':<20}")
        print("-" * 50)
        for doctor in doctors:
            print(f"{doctor[0]:<12}{doctor[1]:<20}{doctor[2]:<20}")

        while True:
            doctor_id = input("Enter Doctor ID to book appointment: ")
            if doctor_id.isdigit() and any(doctor[0] == int(doctor_id) for doctor in doctors):
                doctor_id = int(doctor_id)
                break
            print("Invalid Doctor ID. Please enter a valid ID from the list above.")

        while True:
            appointment_date = input("Enter appointment date (YYYY-MM-DD): ")
            try:
                datetime.strptime(appointment_date, '%Y-%m-%d')
                break
            except ValueError:
                print("Invalid date format. Please enter in YYYY-MM-DD format.")

        while True:
            appointment_time = input("Enter appointment time (HH:MM, 24-hour format): ")
            try:
                datetime.strptime(appointment_time, '%H:%M')
                break
            except ValueError:
                print("Invalid time format. Please enter in HH:MM (24-hour) format.")

        query = """
            INSERT INTO appointments (patient_id, doctor_id, appointment_date, appointment_time, status)
            VALUES (%s, %s, %s, %s, 'pending')
        """
        cursor.execute(query, (patient_id, doctor_id, appointment_date, appointment_time))
        print("Appointment booked successfully with status 'pending'!")
    except Exception as e:
        print(f"Error booking appointment: {e}")

# View patient's treatment plan
def view_treatment_plan(cursor, patient_id):
    try:
        query = """
            SELECT tp.treatment_id, d.name AS doctor_name, tp.diagnosis, tp.medications, tp.procedures, tp.treatment_date
            FROM treatment_plans tp
            JOIN doctors d ON tp.doctor_id = d.doctor_id
            WHERE tp.patient_id = %s
        """
        cursor.execute(query, (patient_id,))
        treatment_plans = cursor.fetchall()

        if not treatment_plans:
            print("No treatment plans found.")
            return

        print("\nTreatment Plans:")
        print(f"{'Treatment ID':<15}{'Doctor Name':<25}{'Diagnosis':<30}{'Medications':<40}{'Procedures':<30}{'Treatment Date':<15}")
        print("-" * 150)
        
        for plan in treatment_plans:
            treatment_id, doctor_name, diagnosis, medications, procedures, treatment_date = plan
            
            # Format treatment date if it's a datetime object
            if treatment_date:
                treatment_date = treatment_date.strftime('%Y-%m-%d')  # Adjust format as needed
            
            # Ensure the length of each printed field accommodates the expected data length
            print(f"{treatment_id:<15}{doctor_name:<25}{diagnosis:<30}{medications:<40}{procedures:<30}{treatment_date:<15}")

    except Exception as e:
        print(f"Error viewing treatment plans: {e}")

# Delete appointment
def delete_appointment(cursor, patient_id):
    try:
        # Query to get all appointments for the patient
        query = """
            SELECT appointment_id, status 
            FROM appointments 
            WHERE patient_id = %s
        """
        cursor.execute(query, (patient_id,))
        appointments = cursor.fetchall()

        if not appointments:
            print("No appointments found.")
            return

        # Display the appointments
        print("\nYour Appointments:")
        print(f"{'Appointment ID':<18}{'Status':<15}")
        print("-" * 40)
        for appointment in appointments:
            appointment_id, status = appointment
            print(f"{appointment_id:<18}{status:<15}")

        # Input for appointment ID
        while True:
            appointment_id = input("Enter the Appointment ID to delete (or 'no' to cancel): ")
            if appointment_id.lower() == 'no':
                print("No appointment deleted.")
                return
            if appointment_id.isdigit() and any(app[0] == int(appointment_id) for app in appointments):
                appointment_id = int(appointment_id)
                break
            print("Invalid Appointment ID. Please enter a valid ID from the list above.")

        # Check appointment status
        cursor.execute("SELECT status FROM appointments WHERE appointment_id = %s", (appointment_id,))
        status = cursor.fetchone()

        if status and status[0] in ['pending', 'accepted']:
            # Delete the appointment
            delete_query = "DELETE FROM appointments WHERE appointment_id = %s"
            cursor.execute(delete_query, (appointment_id,))
            print("Appointment deleted successfully.")
        else:
            print("Cannot delete appointment: only 'pending' or 'accepted' appointments can be deleted.")
    except Exception as e:
        print(f"Error deleting appointment: {e}")
        
def check_appointment_status(cursor, patient_id):
    try:
        query = """
            SELECT a.appointment_id, d.name, a.appointment_date, a.appointment_time, a.status 
            FROM appointments a
            JOIN doctors d ON a.doctor_id = d.doctor_id
            WHERE a.patient_id = %s
        """
        cursor.execute(query, (patient_id,))
        appointments = cursor.fetchall()

        if not appointments:
            print("No appointments found.")
            return

        # Display all appointments with status
        print("\nYour Appointments:")
        print(f"{'Appointment ID':<18}{'Doctor Name':<25}{'Date':<18}{'Time':<12}{'Status':<18}")
        print("-" * 75)
        for appointment in appointments:
            appointment_id, doctor_name, appointment_date, appointment_time, status = appointment

            # Ensure proper formatting of appointment date and time
            if appointment_date and appointment_time:
                appointment_date = appointment_date.strftime('%Y-%m-%d')  # Format date
                appointment_time = appointment_time.strftime('%H:%M')      # Format time
            
                print(f"{appointment_id:<18}{doctor_name:<25}{appointment_date:<18}{appointment_time:<12}{status:<18}")
            else:
                print("Error: One or more appointment details are missing.")
    except Exception as e:
        print(f"Error checking appointment status: {e}")
        
        
def view_bills(cursor, patient_id):
    try:
        query = """
            SELECT bill_id, patient_id, amount, billing_date, status
            FROM bills
            WHERE patient_id = %s
        """
        cursor.execute(query, (patient_id,))
        bills = cursor.fetchall()

        if not bills:
            print("No bills found for the patient.")
            return

        print("\nBills:")
        print(f"{'Bill ID':<10}{'Patient ID':<12}{'Amount':<10}{'Billing Date':<15}{'Status':<10}")
        print("-" * 80)

        for bill in bills:
            bill_id, patient_id, amount, billing_date, status = bill

            # Check if the billing_date is of date or datetime type
            if isinstance(billing_date, (date, datetime)):
                billing_date_str = billing_date.strftime('%Y-%m-%d')
            else:
                billing_date_str = str(billing_date)

            print(f"{str(bill_id):<10}{str(patient_id):<12}{str(amount):<10}{billing_date_str:<15}{status:<10}")
    except Exception as e:
        print(f"Error viewing bills: {e}")
        
# Delete an appointment
def delete_appointment(cursor, patient_id):
    try:
        # Query to get all appointments for the patient
        query = """
            SELECT appointment_id, status 
            FROM appointments 
            WHERE patient_id = %s
        """
        cursor.execute(query, (patient_id,))
        appointments = cursor.fetchall()

        if not appointments:
            print("No appointments found.")
            return

        # Display the appointments
        print("\nYour Appointments:")
        print(f"{'Appointment ID':<18}{'Status':<15}")
        print("-" * 40)
        for appointment in appointments:
            appointment_id, status = appointment
            print(f"{appointment_id:<18}{status:<15}")

        # Input validation for appointment ID
        while True:
            appointment_id = input("Enter the Appointment ID to delete: ")
            if appointment_id.isdigit() and any(app[0] == int(appointment_id) for app in appointments):
                appointment_id = int(appointment_id)
                break
            print("Invalid Appointment ID. Please enter a valid ID from the list above.")

        # Check appointment status
        cursor.execute("SELECT status FROM appointments WHERE appointment_id = %s", (appointment_id,))
        status = cursor.fetchone()

        if status and status[0] in ['pending', 'accepted']:
            # Delete the appointment
            delete_query = "DELETE FROM appointments WHERE appointment_id = %s"
            cursor.execute(delete_query, (appointment_id,))
            print("Appointment deleted successfully.")
        else:
            print("Cannot delete appointment: only 'pending' or 'accepted' appointments can be deleted.")
    except Exception as e:
        print(f"Error deleting appointment: {e}")
        


# Main function to run the program
def main():
    conn = connect_db()
    if conn is None:
        return

    cursor = conn.cursor()
    while True:
        print("\nWelcome To Patients Dashboard")
        print("1. Register Patient")
        print("2. Patient Login")
        print("3. Exit")
        choice = input("Choose an option: ")

        if choice == '1':
            register_patient(cursor)
        elif choice == '2':
            patient = login_patient(cursor)
            if patient:
                patient_id = patient[0]  # Assuming patient_id is the first column
                while True:
                    print("\n1. Book Appointment")
                    print("2. View Treatment Plan")
                    print("3. Delete Appointment")
                    print("4. Update Patient Details")
                    print("5. Check appointment status")
                    print("6. View Bills")
                    print("7. Logout")
                    action_choice = input("Choose an action: ")

                    if action_choice == '1':
                        book_appointment(cursor, patient_id)
                    elif action_choice == '2':
                        view_treatment_plan(cursor, patient_id)
                    elif action_choice == '3':
                        delete_appointment(cursor, patient_id)
                    elif action_choice == '4':
                        update_patient(cursor, patient_id)
                    elif action_choice == '5':
                        check_appointment_status(cursor, patient_id)
                    elif action_choice == '6':
                        view_bills(cursor, patient_id)  
                    elif action_choice == '7':
                        print("Logging out")
                        break
                    else:
                        print("Invalid choice. Please try again.")
        elif choice == '3':
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please try again.")

    cursor.close()
    conn.close()

if __name__ == "__main__":
    main()



Welcome To Patients Dashboard
1. Register Patient
2. Patient Login
3. Exit
Choose an option: 2
Enter your patient ID: 7
Enter your name: ro
Invalid patient ID or name.

Welcome To Patients Dashboard
1. Register Patient
2. Patient Login
3. Exit
Choose an option: 1
Enter your name: kittu
Enter your age: 41
Enter your gender: male
Enter your contact number: 7485961230
Enter your address: ettimadai
Enter your admission date (YYYY-MM-DD): 2024-10-22
Registration successful!
Patient ID: 8, Name: kittu

Welcome To Patients Dashboard
1. Register Patient
2. Patient Login
3. Exit
Choose an option: 2
Enter your patient ID: 8
Enter your name: kittu
Welcome back, kittu!

1. Book Appointment
2. View Treatment Plan
3. Delete Appointment
4. Update Patient Details
5. Check appointment status
6. View Bills
7. Logout
Choose an action: 2
No treatment plans found.

1. Book Appointment
2. View Treatment Plan
3. Delete Appointment
4. Update Patient Details
5. Check appointment status
6. View Bills
7. Logout